In [52]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time

import scipy
import numpy as np
from scipy import linalg
import os
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import time
import copy
from scipy import signal
# import necessary stuff from quidi
from logic.pulsed.pulse_objects import PulseBlock, PulseBlockEnsemble, PulseSequence

# Functions

## FFT

In [53]:
def get_ft_windows():
    """ Retrieve the available windows to be applied on signal data before FT.

    @return: dict with keys being the window name and items being again a dict
             containing the actual function and the normalization factor to
             calculate correctly the amplitude spectrum in the Fourier Transform

    To find out the amplitude normalization factor check either the scipy
    implementation on
        https://github.com/scipy/scipy/blob/v0.15.1/scipy/signal/windows.py#L336
    or just perform a sum of the window (oscillating parts of the window should
    be averaged out and constant offset factor will remain):
        MM=1000000  # choose a big number
        print(sum(signal.hanning(MM))/MM)
    """

    win = {'none': {'func': np.ones, 'ampl_norm': 1.0},
           'hamming': {'func': signal.hamming, 'ampl_norm': 1.0/0.54},
           'hann': {'func': signal.hann, 'ampl_norm': 1.0/0.5},
           'blackman': {'func': signal.blackman, 'ampl_norm': 1.0/0.42},
           'triang': {'func': signal.triang, 'ampl_norm': 1.0/0.5},
           'flattop': {'func': signal.flattop, 'ampl_norm': 1.0/0.2156},
           'bartlett': {'func': signal.bartlett, 'ampl_norm': 1.0/0.5},
           'parzen': {'func': signal.parzen, 'ampl_norm': 1.0/0.375},
           'bohman': {'func': signal.bohman, 'ampl_norm': 1.0/0.4052847},
           'blackmanharris': {'func': signal.blackmanharris, 'ampl_norm': 1.0/0.35875},
           'nuttall': {'func': signal.nuttall, 'ampl_norm': 1.0/0.3635819},
           'barthann': {'func': signal.barthann, 'ampl_norm': 1.0/0.5}}
    return win

def compute_ft(x_val, y_val, zeropad_num=0, window='none', base_corr=True, psd=False):
    """ Compute the Discrete fourier Transform of the power spectral density

    @param numpy.array x_val: 1D array
    @param numpy.array y_val: 1D array of same size as x_val
    @param int zeropad_num: optional, zeropadding (adding zeros to the end of
                            the array). zeropad_num >= 0, the size of the array
                            which is add to the end of the y_val before
                            performing the Fourier Transform (FT). The
                            resulting array will have the length
                                (len(y_val)/2)*(zeropad_num+1)
                            Note that zeropadding will not change or add more
                            information to the dft, it will solely interpolate
                            between the dft_y values (corresponds to a Sinc
                            interpolation method).
                            Set zeropad_num=1 to obtain output arrays which
                            have the same size as the input arrays.
                            Default is zeropad_num=0.
    @param str window: optional, the window function which should be applied to
                       the y values before Fourier Transform is calculated.
    @param bool base_corr: Select whether baseline correction shoud be performed
                           before calculating the FT.
    @param bool psd: optional, select whether the Discrete Fourier Transform or
                     the Power Spectral Density (PSD, which is just the FT of
                     the absolute square of the y-values) should be computed.
                     Default is psd=False.

    @return: tuple(dft_x, dft_y):
                be aware that the return arrays' length depend on the zeropad
                number like
                    len(dft_x) = len(dft_y) = (len(y_val)/2)*(zeropad_num+1)

    Pay attention that the return values of the FT have only half of the
    entries compared to the used signal input (if zeropad=0).

    In general, a window function should be applied on the y data before
    calculating the FT, to reduce spectral leakage. The Hann window for
    instance is almost never a bad choice. Use it like window='hann'

    Keep always in mind the relation to the Fourier transform space:
        T = delta_t * N_samples
    where delta_t is the distance between the time points and N_samples are the
    amount of points in the time domain. Consequently the sample rate is
        f_samplerate = T / N_samples

    Keep also in mind that the FT returns value from 0 to f_samplerate, or
        equivalently -f_samplerate/2 to f_samplerate/2.

    Difference between PSD and DFT:
    The power spectral density (PSD) describes how the power of your signal is
    distributed over frequency whilst the DFT shows the spectral content of
    your signal, i.e. the amplitude and phase of harmonics in your signal.
    """

    avail_windows = get_ft_windows()

    x_val = np.array(x_val)
    y_val = np.array(y_val)

    # Make a baseline correction to avoid a constant offset near zero
    # frequencies. Offset of the y_val from mean corresponds to half the value
    # at fft_y[0].
    corrected_y = y_val
    if base_corr:
        corrected_y = y_val - y_val.mean()

    ampl_norm_fact = 1.0
    # apply window to data to account for spectral leakage:
    if window in avail_windows:
        window_val = avail_windows[window]['func'](len(y_val))
        corrected_y = corrected_y * window_val
        # to get the correct amplitude in the amplitude spectrum
        ampl_norm_fact = avail_windows[window]['ampl_norm']

    # zeropad for sinc interpolation:
    zeropad_arr = np.zeros(len(corrected_y)*(zeropad_num+1))
    zeropad_arr[:len(corrected_y)] = corrected_y

    # Get the amplitude values from the fourier transformed y values.
    fft_y = np.abs(np.fft.fft(zeropad_arr))

    # Power spectral density (PSD) or just amplitude spectrum of fourier signal:
    power_value = 1.0
    if psd:
        power_value = 2.0

    # The factor 2 accounts for the fact that just the half of the spectrum was
    # taken. The ampl_norm_fact is the normalization factor due to the applied
    # window function (the offset value in the window function):
    fft_y = ((2/len(y_val)) * fft_y * ampl_norm_fact)**power_value

    # Due to the sampling theorem you can only identify frequencies at half
    # of the sample rate, therefore the FT contains an almost symmetric
    # spectrum (the asymmetry results from aliasing effects). Therefore take
    # the half of the values for the display.
    middle = int((len(zeropad_arr)+1)//2)

    # sample spacing of x_axis, if x is a time axis than it corresponds to a
    # timestep:
    x_spacing = np.round(x_val[-1] - x_val[-2], 12)

    # use the helper function of numpy to calculate the x_values for the
    # fourier space. That function will handle an occuring devision by 0:
    fft_x = np.fft.fftfreq(len(zeropad_arr), d=x_spacing)

    return abs(fft_x[:middle]), fft_y[:middle]

## Data normalization: Get offset, Amplitude etc

In [54]:
def data_norm(tau_array,E):
    #returns offset, amplitude, phase and frequency of a fit of given data E
    
    # convert to natural time scale
    tau_array = tau_array * 10**9

    # define the expected functions
    def rabi(x, alpha, beta, phi, w):#, T, n):
         return  alpha + beta * np.cos(w * x+ phi)# * np.exp(-( x / T)**n)

    # calculate the guess parameters
    beta_guess = (np.max(E) - np.min(E)) / 2
    alpha_guess = np.mean(E)
    
    # guess frequency
    [fft_x,fft_y]=compute_ft(tau_array, E, zeropad_num=0, window='none', base_corr=True, psd=False)
    guess_w = fft_x[np.argmax(fft_y)]

    # guess phase
    tau_step = tau_array[1] - tau_array[0]
    sampling_freq = 1 / tau_step

    period = 1/guess_w
    period_id = int(np.around(period * sampling_freq,0))
    first_max = np.argmax(E[0:period_id])
    phase_guess = 2 * np.pi * tau_array[first_max] / period

    # guess parameters
    guess_vals = [alpha_guess, beta_guess, -phase_guess, guess_w * 2 * np.pi]#, 100e3, 2]

    #fitting
    fit_params, cov_mat = curve_fit(rabi, tau_array, E, p0 = guess_vals) #
    fit_errors = np.sqrt(np.diag(cov_mat)) # stderr of the parameters 
    
    fit_residual = E - rabi(tau_array, *fit_params) 
    fit_Rsquared = 1 - np.var(fit_residual)/np.var(E)
    #print('Fit R-squared_x', fit_Rsquared, '\n')
    
    return fit_params

## Calculate the parameterized density matrix

In [55]:
def rho_parameterized(a,b):
    # calculate the parameterized density matrix, a=nu b=xi
    
    val_1 = 0.5 * (1 + np.cos(a) * np.cos(b))
    
    val_2 = 0.5 * (np.cos(b) * np.sin(a) + 1j * np.sin(b))
    
    val_3 = 0.5 * (np.cos(b) * np.sin(a) - 1j * np.sin(b))
    
    val_4 = 0.5 * (1 - np.cos(a) * np.cos(b))
    
    rho_para = np.array([[val_1, val_2],[val_3, val_4]],dtype=complex)
    
    return rho_para

## Calculate density matrix from measurements

In [56]:
def calculate_rho(rho_data):

    rho_return = np.zeros([2,2], dtype = complex)
    rho_return[0,0] = 1/2 + 1/2 * (rho_data[0] - rho_data[1])
    rho_return[0,1] = 1/2 * (rho_data[3] + 1j * rho_data[2])
    rho_return[1,0] = 1/2 * (rho_data[3] - 1j * rho_data[2]) 
    rho_return[1,1] = 1/2 - 1/2 * (rho_data[0] - rho_data[1])

    return rho_return

## find a physically correct density matrix

In [57]:
def rho_min(rho):
    #find a physically correct density matrix
    
    sigma = np.zeros((2,2))
    # substitute constant values; rho[0,0]) and rho[1,1] are always real!
    a = np.real(1 + (-1 + rho[0,0]) * rho[0,0] + (-1 + rho[1,1]) * rho[1,1] + 2 * np.abs(rho[0,1]))**2
    b = np.real(-rho[0,0] + rho[1,1])
    c = -2 * np.real(rho[0,1])
    d = - 2 * np.imag(rho[0,1])

    n1 = np.arctan2([-c] , [-b])
    n1 = n1[0]
    n2 = np.arctan2([c], [b])
    n2 = n2[0]
    x1 = -np.arccos(- np.sqrt((b**2 + c**2)/(b**2 + c**2 + d**2)))
    x2 = np.arccos(- np.sqrt((b**2 + c**2)/(b**2 + c**2 + d**2)))

    sigma[0,0] = a + b * np.cos(n1) * np.cos(x1) + c * np.cos(x1) * np.sin(n1) + d * np.sin(x1)
    sigma[0,1] = a + b * np.cos(n1) * np.cos(x2) + c * np.cos(x2) * np.sin(n1) + d * np.sin(x2)
    sigma[1,0] = a + b * np.cos(n2) * np.cos(x1) + c * np.cos(x1) * np.sin(n2) + d * np.sin(x1)
    sigma[1,1] = a + b * np.cos(n2) * np.cos(x2) + c * np.cos(x2) * np.sin(n2) + d * np.sin(x2)

    pos_min = np.unravel_index(np.argmin(sigma), np.shape(sigma))
    n_min = int(pos_min[0])
    x_min = int(pos_min[1])
  
    if n_min == 0 and x_min == 0:
        rho_return = rho_parameterized(n1,x1)
    elif n_min == 0 and x_min == 1:
        rho_return = rho_parameterized(n1,x2)
    elif n_min == 1 and x_min == 0:
        rho_return = rho_parameterized(n2,x1)
    elif n_min == 1 and x_min == 1:
        rho_return = rho_parameterized(n2,x2)
        
    return rho_return

## Quantum State Tomography

In [58]:
def calc_chi(rho0_chi,rho1_chi,rho2_chi,rho3_chi):
    #"Quantum State Tomography: calculate matrix Chi from given density matrices rho1,..,rho4"
    sigx = np.array([[0,1],[1,0]])

    #Defintion of Matrix M

    M=np.zeros([8,8],dtype=complex)
    M[0,0]=0
    M[0,1]=0
    M[0,2]=0
    M[0,3]=0
    M[0,4]=0
    M[0,5]=0
    M[0,6]=1
    M[0,7]=0

    M[1,0]=0
    M[1,1]=0
    M[1,2]=0
    M[1,3]=0
    M[1,4]=0
    M[1,5]=0
    M[1,6]=0
    M[1,7]=1

    M[2,0]=1
    M[2,1]=0
    M[2,2]=0
    M[2,3]=0
    M[2,4]=0
    M[2,5]=0
    M[2,6]=0
    M[2,7]=0

    M[3,0]=0
    M[3,1]=1
    M[3,2]=0
    M[3,3]=0
    M[3,4]=0
    M[3,5]=0
    M[3,6]=0
    M[3,7]=0

    M[4,0]=0.5
    M[4,1]=0
    M[4,2]=-0.5j
    M[4,3]=0
    M[4,4]=0.5j
    M[4,5]=0
    M[4,6]=0.5
    M[4,7]=0

    M[5,0]=0
    M[5,1]=0.5
    M[5,2]=0
    M[5,3]=-0.5j
    M[5,4]=0
    M[5,5]=0.5j
    M[5,6]=0
    M[5,7]=0.5

    M[6,0]=0.5
    M[6,1]=0
    M[6,2]=-0.5
    M[6,3]=0
    M[6,4]=-0.5
    M[6,5]=0
    M[6,6]=0.5
    M[6,7]=0

    M[7,0]=0
    M[7,1]=0.5
    M[7,2]=0
    M[7,3]=-0.5
    M[7,4]=0
    M[7,5]=-0.5
    M[7,6]=0
    M[7,7]=0.5

    #calculate inverse of matrix M
    Minv = np.linalg.inv(M)
    
    #convert matrices rho1,...,rho4 to one 8x2 matrix
    rho_chi=np.zeros([8,2],dtype=complex)
    rho_chi[0,0]=rho0_chi[0,0]
    rho_chi[0,1]=rho0_chi[0,1]
    rho_chi[1,0]=rho0_chi[1,0]
    rho_chi[1,1]=rho0_chi[1,1]

    rho_chi[2,0]=rho1_chi[0,0]
    rho_chi[2,1]=rho1_chi[0,1]
    rho_chi[3,0]=rho1_chi[1,0]
    rho_chi[3,1]=rho1_chi[1,1]

    rho_chi[4,0]=rho2_chi[0,0]
    rho_chi[4,1]=rho2_chi[0,1]
    rho_chi[5,0]=rho2_chi[1,0]
    rho_chi[5,1]=rho2_chi[1,1]

    rho_chi[6,0]=rho3_chi[0,0]
    rho_chi[6,1]=rho3_chi[0,1]
    rho_chi[7,0]=rho3_chi[1,0]
    rho_chi[7,1]=rho3_chi[1,1]
    
    #Minv times rho
    rhoprime = np.matmul(Minv,rho_chi)
    #convert rhoprime (8x2) into a 4x4 matrix
    rhoM=np.zeros([4,4],dtype=complex)
    rhoM[0:2,0:2]=rhoprime[0:2,0:2]
    rhoM[0:2,2:4]=rhoprime[2:4,0:2]
    rhoM[2:4,0:2]=rhoprime[4:6,0:2]
    rhoM[2:4,2:4]=rhoprime[6:9,0:2]
    
    #Definition of Lambda
    Lambda=np.zeros([4,4],dtype=complex)
    Lambda[0:2,0:2]=np.identity(2)
    Lambda[2:4,0:2]=sigx
    Lambda[0:2,2:4]=sigx
    Lambda[2:4,2:4]=-np.identity(2)
    Lambda = Lambda * 0.5
    
    #calculate Chi
    chi = np.matmul(Lambda,np.matmul(rhoM,Lambda))                    
    
    return chi

## Figure of merit

In [59]:
def FoM(chi):
    #"calculate fidelity of Sx pi-pulse"
    #Definition of pauli and spin matrices
    sigx = np.array([[0,1],[1,0]])
    sigy = np.array([[0,-1j],[1j,0]])
    sigz = np.array([[1,0],[0,-1]])

    Sx = 0.5 * sigx
    Sy = 0.5 * sigy
    Sz= 0.5 * sigz

    #initial state
    init1 = np.array([[0,0],[0,1]])
    init2 = np.array([[1,0],[0,0]])
    init3 = np.matmul(np.matmul(linalg.expm(-1j*np.pi/2*Sx),init1),linalg.expm(1j*np.pi/2*Sx))
    init4 = np.matmul(np.matmul(linalg.expm(-1j*np.pi/2*Sy),init1),linalg.expm(1j*np.pi/2*Sy))

    #time evolution operator
    U = linalg.expm(-1j * np.pi * Sx)

    #final state
    final1 = np.matmul(np.matmul(U,init1),U.conj().transpose())
    final2 = np.matmul(np.matmul(U,init2),U.conj().transpose())
    final3 = np.matmul(np.matmul(U,init3),U.conj().transpose())
    final4 = np.matmul(np.matmul(U,init4),U.conj().transpose())
    
    #calculate theoretical chi of Sx pi-pulse
    chi_theo = calc_chi(final1,final2,final3,final4)
    
    # calculate figure of merit = gate fidelity
    #fom = np.real(np.trace(np.matmul(chi_theo.conj().transpose(),chi)))
    fom = 1 - 1/4 * np.trace(np.sqrt(np.matmul((chi-chi_theo).conj().transpose(),(chi - chi_theo))))
    
    return fom

## Contrast of Rabi

In [60]:
def contrast_rabi(tau_array,data):
    #claclulates denity matrix rho from data sets E_x and E_y (Rabi)
    # convert to natural time scale
    tau_array = tau_array * 10**9

    # define the expected functions
    def rabi(x, alpha, beta, phi, w, T):
         return  alpha + beta * np.cos(w * x+ phi)* np.exp(-( x / T))
        
    # calculate the guess parameters
    beta_guess_x = (np.max(data) - np.min(data)) / 2
    alpha_guess_x = np.mean(data)
    beta_guess_min_x = 0
    beta_guess_max_x = 10
    
    # guess frequency
    [fft_x_x,fft_y_x]=compute_ft(tau_array, data, zeropad_num=0, window='none', base_corr=True, psd=False)
    guess_w_x = fft_x_x[np.argmax(fft_y_x)]
    
    # guess phase
    tau_step = tau_array[1] - tau_array[0]
    sampling_freq = 1 / tau_step

    period_x = 1/guess_w_x
    period_idx_x = int(np.around(period_x * sampling_freq,0))
    first_max_x = np.argmax(data[0:period_idx_x])
    phase_guess_x = 2 * np.pi * tau_array[first_max_x] / period_x
    
    # limit phase betwen -pi/2 and +pi/2
    if phase_guess_x > np.pi/2 and phase_guess_x <= 3 * np.pi / 2:
        phase_guess_x = phase_guess_x - np.pi
        beta_guess_x = - beta_guess_x
        beta_guess_min_x = -1
        beta_guess_max_x = 0
    
    elif  phase_guess_x > 3 * np.pi / 2 and phase_guess_x <= 2 * np.pi:
        phase_guess_x = phase_guess_x - 2 * np.pi

    # guess values for exp decay
    guess_T = 720
    
    # boundaris for exp dexay
    bound_T_min = 0
    bound_T_max = 2000

    # guess parameters
    guess_vals_x = [alpha_guess_x, beta_guess_x, -phase_guess_x, guess_w_x * 2 * np.pi, guess_T]

    # parameters bounds, negative beta needs to be possible, phi between -pi/2 nad pi/2 with exp decay
    param_bounds_x = ([0, beta_guess_min_x, -np.pi / 2, 0, bound_T_min],[1000 , beta_guess_max_x, np.pi / 2, 1.2 * guess_w_x * 2 * np.pi, bound_T_max])
    
    #fitting
    fit_params_x, cov_mat_x = curve_fit(rabi, tau_array, data, p0 = guess_vals_x, bounds = param_bounds_x, maxfev=100000) #
    fit_errors_x = np.sqrt(np.diag(cov_mat_x)) # stderr of the parameters
    #######################################################################################################################    
    # manually calculate R-squared goodness of fit
    #fit_residual_x = data - rabi(tau_array, *fit_params_x) 
    #fit_Rsquared_x = 1 - np.var(fit_residual_x)/np.var(data)
    
    offset = fit_params_x[0]
    amplitude = fit_params_x[1]
    contrast=1-(offset - amplitude)/(offset + amplitude)
    
    return contrast

# QuOCS and Qudi with Noise

## **What to do before**

* Update the QuOCS library
* Substitute the logic files in Qudi

To handle the noise in the physical system with QuOCS, we will use the **DCRABNoisyAlgorithm** class.
In particular, we will use the compensation drift functionality to reset the figure of merit at the beginning of every Super iteration to take into account any new drift term coming from the external noise sources.
Furthermore, a re-evaluation step method is added from the second iteration for every super-iteration. Indeed the measure is repeated n + 1 times, where n is the length of the probabilities list.

The changes to be made are in the algorithm settings and in the evaluation loop (See below). Both changes are marked in bold.

### Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization. 

In [61]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

**CHANGE** :Now we will use the **dCRAB Noisy algorithm** which ensures **drift compensation** and handles **fluctuations** 

In [62]:
# optimization_dictionary = {"optimization_client_name": optimization_client_name,
#                            'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABAlgorithm', 
#                            'opti_algorithm_class': 'DCrabAlgorithm', 
#                           }

# DCrabNoisyAlgorithm

optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 
                           'opti_algorithm_class': 'DCrabNoisyAlgorithm', 
                          }

Number of iteration and super-iterations

In [63]:
# Total number of dCRAB superiteration
super_iteration_number = 4
# Maximum number of iteration per super-iteration
maximum_function_evaluations_number = 500

To activate the drift compensation and the re-evaluation steps add 

In [64]:
optimization_dictionary['algorithm_settings'] = {"algorithm_name": "dCRAB",
                                                 "super_iteration_number": super_iteration_number,
                                                 "max_eval_total": super_iteration_number * maximum_function_evaluations_number,
                                                 "FoM_goal": 0.00001,
                                                 "total_time_lim": 3,
                                                 "compensate_drift": {
                                                     "compensate_after_SI": True,
                                                     "compensate_after_minutes": 15
                                                 },
                                                 "random_number_generator":{
                                                     "seed_number":420
                                                 }
}

Settings for the inner algorithm used by dCRAB

In [65]:
optimization_dictionary['algorithm_settings']["dsm_settings"] = {'general_settings': {
                                                "dsm_algorithm_name": "NelderMead", 
                                                'is_adaptive': False
                                            }, 
                                            'stopping_criteria': {
                                                "max_eval": 100,
                                                "time_lim": 45,
                                                "xatol": 1e-10, 
                                                "frtol": 1e-10}
                                            }

### Times

We can define here how many times we need to use in the optimization

In [66]:
time_p = {'time_name': 'time_p', 'initial_value': 201e-9} # old: 201e-9

In [67]:
optimization_dictionary["times"] = [time_p]

### Parameters

In [68]:
optimization_dictionary['parameters'] = []

### Pulses

In [69]:
pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 0.5, 
                   'lower_limit': -0.5, 
                   'bins_number': 1001, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.04, # added to guess amp, to create 3 points for start simplex
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': 5, # number of frequencies within the below defined range
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.01, 'upper_limit': 30.0} # number of oscillations within pulse
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0331175 + 0.0*t'}
                  }

# 'amplitude_variation': 0.04
# lambda t: 0.0325 + 0.0*t    0.0331175
# number of osc: 7

In [70]:
# 200 ns: 0.0325

In [71]:
pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 10, 
               'lower_limit': -10, 
               'bins_number': 1001, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': 3, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': -10, 'upper_limit': 10.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

In [72]:
optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [73]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}

Load the optimization algorithm into the optimization logic and display it into the GUI

In [74]:
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

In [75]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'optimization_client_name': 'test_dCRAB_Noisy_2_control_fields', 'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 'opti_algorithm_class': 'DCrabNoisyAlgorithm', 'algorithm_settings': {'algorithm_name': 'dCRAB', 'super_iteration_number': 4, 'max_eval_total': 2000, 'FoM_goal': 1e-05, 'total_time_lim': 3, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'max_eval': 100, 'time_lim': 45, 'xatol': 1e-10, 'frtol': 1e-10}}}, 'times': [{'time_name': 'time_p', 'initial_value': 2.01e-07}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 0.5, 'lower_limit': -0.5, 'bins_number': 1001, 'time_name': 'time_p', 'amplitude_variation': 0.04, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quocslib.p

**CHANGE** : We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stays for the experimental standard deviation.

# Quantum Process Tomography

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# file paths 
folder_path = r'C:\Users\Mesoscopic\Desktop\quocs_pulses'
filename_amplitude=r'amplitude'
filename_phase=r'phase'

# runtime of the experiment
runtime = 5

length_oc = 200e-9

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.2827e9
globaldict["rabi_period"] = 57.22e-9
globaldict["laser_length"] = 25e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 0)
pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": True}
qubit = OneQubit2Fields(args_dict)


for vec_number in [2,3,4,6,8]:
    print(vec_number)
    save_path=r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results\vec'+ str(vec_number)

    
    pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 0.5, 
                   'lower_limit': -0.5, 
                   'bins_number': 1001, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.04, # added to guess amp, to create 3 points for start simplex
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': vec_number, # number of frequencies within the below defined range
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.01, 'upper_limit': 30.0} # number of oscillations within pulse
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0331175 + 0.0*t'}
                  }
    pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 10, 
               'lower_limit': -10, 
               'bins_number': 1001, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': vec_number, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': -10, 'upper_limit': 10.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }
    optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]
    opti_comm_dict = {"optimization_dictionary": optimization_dictionary}
    # Load the optimization algorithm into the optimization logic and display it into the GUI
    optimizationlogic.load_opti_comm_dict(opti_comm_dict)

    ######################################################################################################
    # Measurement rect
    ######################################################################################################

    # predefined method
    dictparamsall = pulsedmasterlogic.generate_method_params
    ############################ Uploading all predefined methods needed ###################################################
    # predefined method
    dict_qst_rec = dictparamsall["qst_state_rec"]
    dict_qst_rec["name"] = 'qststaterec'
    dict_qst_rec["length"] = length_oc
    dict_qst_rec["amplitude"] = globaldict["microwave_amplitude"]

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('qst_state_rec',kwarg_dict = dict_qst_rec)  

    time.sleep(1)

    # upload the sequence
    pulsedmasterlogic.sample_ensemble(dict_qst_rec["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
        time.sleep(1)

    # start the measurement
    #print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)

    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()

    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)

        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break

    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break

    #time.sleep(1)

    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)

    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    #######################################################################################################
    # Analysis rect
    ######################################################################################################
    signal_data = pulsedmeasurementlogic.signal_data

    E1 = signal_data[1,2:6]
    E0 = signal_data[1,6:10]
    E2 = signal_data[1,10:14]
    E3 = signal_data[1,14:18]

    # normalize the rabi measurments with m_s=0 and m_s=-1 state measurement
    norm_max = signal_data[1,0]
    norm_min = signal_data[1,1]

    E0 = (E0 - norm_min) / (norm_max-norm_min)
    E1 = (E1 - norm_min) / (norm_max-norm_min)
    E2 = (E2 - norm_min) / (norm_max-norm_min)
    E3 = (E3 - norm_min) / (norm_max-norm_min)

    # calculate the density matrices
    rho0 = calculate_rho(E0)
    rho1 = calculate_rho(E1)
    rho2 = calculate_rho(E2)
    rho3 = calculate_rho(E3)

    # quadratic maximum likelihood estimation of the density matrices
    rho0_est = rho_min(rho0)
    rho1_est = rho_min(rho1)
    rho2_est = rho_min(rho2)
    rho3_est = rho_min(rho3)

    # calculate chi matrix (Qunatum State Tomography)
    chi = calc_chi(rho0_est,rho1_est,rho2_est,rho3_est)

    # calculate the fom
    fom_rec = np.real(1 - FoM(chi))

    np.savetxt(save_path+r'\fom_rec.txt',np.array([fom_rec]))
    
    # just for safety
    time.sleep(5)

    ######################################################################################################
    # Measurement
    ######################################################################################################
    
    # TR_2022_04_12: Here we start the optimization
    
    optimalcontrol.start_optimization()

    # Just a time to check for latent time
    last_time_fom = time.time()
    # repeat the whole process until its manually stopped or QuOCS finsihed the optimization
    # Wait few seconds before starting to get and return data
    while not optimizationlogic.handle_exit_obj.is_user_running:
        time.sleep(0.1)
        if (time.time() - last_time_fom) > 30:
            print("Problem at the beginning! Surpassed the 30 secs")
            break

    # iteration number
    it_val = 0

    # when did the optimization start?
    opt_start_time = time.time()

    # print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
    
    # TR_2022_04_12: The loop runs until the is_user_running in the handle_exit_obj is False
    
    while  optimizationlogic.handle_exit_obj.is_user_running == True:
        time_stamp=time.time()
        # wait until QuOCS optimizes the controls
        # print("Wait until the controls logic gives the controls")
        while not controlslogic.are_pulses_calculated:
            time.sleep(0.1)
            # If the waiting time exceed 10 seconds left stop the optimization
            if time.time() - last_time_fom > 20:
                print("Too much time... Exit!")
                optimizationlogic.handle_exit_obj.is_user_running = False
                break

        # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
        # and status code -1, to interrupt the optimization smoothly
        if optimizationlogic.handle_exit_obj.is_user_running == False:
            print('Stopping the measurement!')
            # Update the pulse with the maximum number
            fomlogic.update_fom(10**10, status_code=-1)
            break

        #######################################################################################################
        # Get the Controls
        #######################################################################################################
        # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
        controlslogic.are_pulses_calculated = False
        # Get the controls from the controls logic
        pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
        #######################################################################################################
        # Perform the measurement
        #######################################################################################################

        # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
        # parameter)
        np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\amplitude.txt',np.column_stack((timegrids[0], pulses[0])))
        np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\phase.txt',np.column_stack((timegrids[0], pulses[1])))
        time.sleep(0.2)

        # predefined method
        dictparamsall = pulsedmasterlogic.generate_method_params
        ############################ Uploading all predefined methods needed ###################################################
        # predefined method
        dict_qst = dictparamsall["qst_state_oc"]
        dict_qst["name"] = 'qststateoc'
        dict_qst["length"] = length_oc
        dict_qst["filename_amplitude"] = filename_amplitude
        dict_qst["filename_phase"] = filename_phase
        dict_qst["folder_path"] = folder_path

        # generate the sequence
        pulsedmasterlogic.generate_predefined_sequence('qst_state_oc',kwarg_dict = dict_qst)  

        time.sleep(1)

        # upload the sequence
        pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
        while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
            time.sleep(1)

        #print('Finished uploading', dictparams_oc["name"], '!')

        # make sure everything is finished (crucial)
        #time.sleep(1)#time.sleep(5)    

         # start the measurement
        #print('Starting the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(True)
        time.sleep(1)

        # make sure the measurement started
        while not pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        measurement_start_time = time.time()

        while time.time() <= measurement_start_time + runtime:
            time.sleep(0.2)

            # option to stop the measurement
            if pulsedmasterlogic.globalrun == False:
                print('Stopping the measurement!')
                break

        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            pulsedmasterlogic.toggle_pulsed_measurement(False)
            break

        #time.sleep(1)

        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)

        # Make sure it stopped (Wait until the Picoscope card sends the data)
        while pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        #######################################################################################################
        # Analysis
        ######################################################################################################
        signal_data = pulsedmeasurementlogic.signal_data

        E1 = signal_data[1,2:6]
        E0 = signal_data[1,6:10]
        E2 = signal_data[1,10:14]
        E3 = signal_data[1,14:18]

        # normalize the rabi measurments with m_s=0 and m_s=-1 state measurement
        norm_max = signal_data[1,0]
        norm_min = signal_data[1,1]

        E0 = (E0 - norm_min) / (norm_max-norm_min)
        E1 = (E1 - norm_min) / (norm_max-norm_min)
        E2 = (E2 - norm_min) / (norm_max-norm_min)
        E3 = (E3 - norm_min) / (norm_max-norm_min)

        # calculate the density matrices
        rho0 = calculate_rho(E0)
        rho1 = calculate_rho(E1)
        rho2 = calculate_rho(E2)
        rho3 = calculate_rho(E3)

        # quadratic maximum likelihood estimation of the density matrices
        rho0_est = rho_min(rho0)
        rho1_est = rho_min(rho1)
        rho2_est = rho_min(rho2)
        rho3_est = rho_min(rho3)

        # calculate chi matrix (Qunatum State Tomography)
        chi = calc_chi(rho0_est,rho1_est,rho2_est,rho3_est)

        # calculate the fom
        fom = np.real(1 - FoM(chi))

        # save the figure of merit to a file for redcrab
        #np.savetxt(FoM_path, np.array([FoM(chi)]), delimiter=',')

        # save the fom to plot its evolution later
        fom_all.append(fom)

        #######################################################################################################
        # Calculate the figure of merit and the standard deviation
        #######################################################################################################
        # calculate the Figure of Merit
        #om_dict = qubit.get_FoM(pulses, timegrids, parameters)
        # Extract the fom and std
        #om, std = fom_dict["FoM"], fom_dict["std"]
        #######################################################################################################
        # Return the figure of merit
        #######################################################################################################
        # Update the figure of merit and the standard deviation to the fom logic
        fomlogic.update_fom(fom, 1,status_code=0)
        #fomlogic.update_fom(fom, std, status_code=0)
        # update the last time the fom is calculated
        last_time_fom = time.time()
        #######################################################################################################
        # Optional part
        #######################################################################################################
        # Print the data just for debug purpose
        #print("FoM: {fom}, Std: {std}, status_code: {status_code}".format(fom=fom, std=std, status_code=0))
        #print(time.time()-time_stamp)

    # when did the optimization stop?
    opt_end_time = time.time()

    print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')

    print("Optimization finished")

     #######################################################################################################
    # Save the results
    #######################################################################################################
    # Access to the optimizer object to get info about the optimization
    optimizer_obj = optimizationlogic.optimization_obj
    np.savetxt(save_path+r'\fom_rec.txt',np.array([optimizer_obj.opt_alg_obj.best_FoM]))

    # Best controls
    pulses_list, time_grids_list, parameters_list = optimizer_obj.opt_alg_obj.get_best_controls()
    t_amplitude = time_grids_list[0]
    amplitude = pulses_list[0]
    t_phase = time_grids_list[1]
    phase = pulses_list[1]

    np.savetxt(save_path+r'\amplitude.txt',np.column_stack((t_amplitude, amplitude)))
    np.savetxt(save_path+r'\phase.txt',np.column_stack((t_phase, phase)))
    
    # just for safety
    time.sleep(5)

## Save the results

In [ ]:
# Access to the optimizer object to get info about the optimization
optimizer_obj = optimizationlogic.optimization_obj

In [ ]:
optimizer_obj.opt_alg_obj.get_best_controls()

In [ ]:
optimizer_obj.best_fom

In [ ]:
# Best controls
pulses_list, time_grids_list, parameters_list = optimizer_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]

In [ ]:
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results\amplitude_03042022.txt',np.column_stack((t_amplitude, amplitude)))
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results\phase_03042022.txt',np.column_stack((t_phase, phase)))
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results\fom_all_200ns_03042022.txt',fom_all)

## Measure chi for best pulse

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# file paths 
folder_path = r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results'
filename_amplitude=r'amplitude_max'
# save the time when the file was last modified
#last_save_amplitude = os.stat(amplitude_path).st_mtime

#FoM_path= r'C:\Users\Mesoscopic\Desktop\Redcrab_data\FoM'
# save the time when the file was last modified
#last_save_phase = os.stat(phase_path).st_mtime
filename_phase=r'phase_max'

# runtime of the experiment
runtime = 5

length_oc = 200e-9


# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.2808605e9
globaldict["rabi_period"] = 55.85e-9
globaldict["laser_length"] = 20e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# get all predefined methods
dictparamsall = pulsedmasterlogic.generate_method_params

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 0)
pulsedmasterlogic.set_analysis_settings(signal_end = 4e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')


############################ Uploading all predefined methods needed ###################################################

# predefined method
dict_qst = dictparamsall["qst_state_oc"]
dict_qst["name"] = 'qststateoc'
dict_qst["length"] = length_oc
dict_qst["filename_amplitude"] = filename_amplitude
dict_qst["filename_phase"] = filename_phase
dict_qst["folder_path"] = folder_path

# generate the sequence
pulsedmasterlogic.generate_predefined_sequence('qst_state_oc',kwarg_dict = dict_qst)  

time.sleep(1)

# upload the sequence
pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
    time.sleep(1)

# start the measurement
print('Starting the measurement!')
pulsedmasterlogic.toggle_pulsed_measurement(True)

time.sleep(1)

# make sure the measurement started
while not pulsedmasterlogic.status_dict['measurement_running']:
    continue

#save the current time
measurement_start_time = time.time()

while time.time() <= measurement_start_time + runtime:
    time.sleep(0.1)
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break

# Stop the measurement
pulsedmasterlogic.toggle_pulsed_measurement(False)

time.sleep(3)

#######################################################################################################
# Analysis
######################################################################################################
signal_qst = pulsedmeasurementlogic.signal_data
signal_data = pulsedmeasurementlogic.signal_data

E1 = signal_data[1,2:6]
E0 = signal_data[1,6:10]
E2 = signal_data[1,10:14]
E3 = signal_data[1,14:18]

 # normalize the rabi measurments with m_s=0 and m_s=-1 state measurement
norm_max = signal_data[1,0]
norm_min = signal_data[1,1]

E0 = (E0 - norm_min) / (norm_max-norm_min)
E1 = (E1 - norm_min) / (norm_max-norm_min)
E2 = (E2 - norm_min) / (norm_max-norm_min)
E3 = (E3 - norm_min) / (norm_max-norm_min)

# calculate the density matrices
rho0 = calculate_rho(E0)
rho1 = calculate_rho(E1)
rho2 = calculate_rho(E2)
rho3 = calculate_rho(E3)

# quadratic maximum likelihood estimation of the density matrices
rho0_est = rho_min(rho0)
rho1_est = rho_min(rho1)
rho2_est = rho_min(rho2)
rho3_est = rho_min(rho3)

# calculate chi matrix (Qunatum State Tomography)
chi = calc_chi(rho0_est,rho1_est,rho2_est,rho3_est)
print(FoM(chi))

In [ ]:
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results\chi.txt',chi)

# State to State Transfer

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# file paths 
folder_path = r'C:\Users\Mesoscopic\Desktop\quocs_pulses'
filename_amplitude=r'amplitude'
filename_phase=r'phase'

#FoM_path= r'C:\Users\Mesoscopic\Desktop\Redcrab_data\FoM'

# runtime of the experiment
runtime = 5

length_oc = 40e-9 #200e-9

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.28284e9
globaldict["rabi_period"] = 57.59e-9
globaldict["laser_length"] = 25e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 0)
pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": True}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# iteration number
it_val = 0

# when did the optimization start?
opt_start_time = time.time()
    
# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    time_stamp=time.time()
    # wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            
    # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    if optimizationlogic.handle_exit_obj.is_user_running == False:
        print('Stopping the measurement!')
        # Update the pulse with the maximum number
        fomlogic.update_fom(10**10, status_code=-1)
        break
        
    #######################################################################################################
    # Get the Controls
    #######################################################################################################
    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    #######################################################################################################
    # Perform the measurement
    #######################################################################################################
    
    # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
    # parameter)
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\amplitude.txt',np.column_stack((timegrids[0], pulses[0])))
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\phase.txt',np.column_stack((timegrids[0], pulses[1])))
    time.sleep(0.2)
    
    # predefined method
    dictparamsall = pulsedmasterlogic.generate_method_params
    ############################ Uploading all predefined methods needed ###################################################
    # predefined method
    dict_qst = dictparamsall["sts_oc"]
    dict_qst["name"] = 'stsoc'
    dict_qst["length"] = length_oc
    dict_qst["filename_amplitude"] = filename_amplitude
    dict_qst["filename_phase"] = filename_phase
    dict_qst["folder_path"] = folder_path

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('sts_oc',kwarg_dict = dict_qst)  
    
    time.sleep(1)
    
    # upload the sequence
    pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
        time.sleep(1)

    #print('Finished uploading', dictparams_oc["name"], '!')
         
    # make sure everything is finished (crucial)
    #time.sleep(1)#time.sleep(5)    
    
     # start the measurement
    #print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)
    
    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()
    
    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break

    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break
        
    #time.sleep(1)
            
    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    
    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)
    
    #######################################################################################################
    # Analysis
    ######################################################################################################
    signal_data = pulsedmeasurementlogic.signal_data
    
    # calculate the fom
    up_norm = signal_data[1,0] / signal_data[1,0]
    down_norm = signal_data[1,1] / signal_data[1,0]
    
    fom = np.real(1 - (up_norm - down_norm))
    #fom = signal_data[1,1] - signal_data[1,0] 
    
    # save the figure of merit to a file for redcrab
    #np.savetxt(FoM_path, np.array([FoM(chi)]), delimiter=',')

    # save the fom to plot its evolution later
    fom_all.append(fom)
    
    #######################################################################################################
    # Calculate the figure of merit and the standard deviation
    #######################################################################################################
    # calculate the Figure of Merit
    #om_dict = qubit.get_FoM(pulses, timegrids, parameters)
    # Extract the fom and std
    #om, std = fom_dict["FoM"], fom_dict["std"]
    #######################################################################################################
    # Return the figure of merit
    #######################################################################################################
    # Update the figure of merit and the standard deviation to the fom logic
    fomlogic.update_fom(fom, 1,status_code=0)
    #fomlogic.update_fom(fom, std, status_code=0)
    # update the last time the fom is calculated
    last_time_fom = time.time()
    #######################################################################################################
    # Optional part
    #######################################################################################################
    # Print the data just for debug purpose
    #print("FoM: {fom}, Std: {std}, status_code: {status_code}".format(fom=fom, std=std, status_code=0))
    #print(time.time()-time_stamp)    
    
# when did the optimization stop?
opt_end_time = time.time()

print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')

print("Optimization finished")

### Save result

In [ ]:
# Access to the optimizer object to get info about the optimization
optimizer_obj = optimizationlogic.optimizer_obj

In [ ]:
optimizer_obj

In [ ]:
optimizer_obj.best_fom

In [ ]:
# Best controls
pulses_list, time_grids_list, parameters_list = optimizer_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]

In [ ]:
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\amplitude_40ns_02042022.txt',np.column_stack((t_amplitude, amplitude)))
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\phase_40ns_021042022.txt',np.column_stack((t_phase, phase)))
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\fom_all_40ns_02042022.txt',fom_all)

## Benchmark OC

In [ ]:
amp_scal_step=0.05
num_of_points=21

signal_bench=np.zeros(num_of_points)
ampl_scale=np.arange(1.0, -0.05, step=-amp_scal_step)

######################################################################################################
# Parameters and Settings
######################################################################################################

# file paths 
folder_path = r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\02042022'
filename_amplitude=r'amplitude'
filename_phase=r'phase'

#FoM_path= r'C:\Users\Mesoscopic\Desktop\Redcrab_data\FoM'

# runtime of the experiment
runtime = 5 # was 20

length_oc = 40e-9

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.28284e9
globaldict["rabi_period"] = 57.59e-9
globaldict["laser_length"] = 25e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 0)
pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

for kk in range (num_of_points):
    # predefined method
    dictparamsall = pulsedmasterlogic.generate_method_params
    ############################ Uploading all predefined methods needed ###################################################
    # predefined method
    dict_qst = dictparamsall["benchmark_oc_pulse"]
    dict_qst["name"] = 'ocbenchmark'
    dict_qst["length"] = length_oc
    dict_qst["amp_scal_start"] = kk * 0.05
    dict_qst["amp_scal_step"] = amp_scal_step
    dict_qst["phase"] = 0
    dict_qst["filename_amplitude"] = filename_amplitude
    dict_qst["filename_phase"] = filename_phase
    dict_qst["folder_path"] = folder_path
    dict_qst["num_of_points"] = 1

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('benchmark_oc_pulse',kwarg_dict = dict_qst)  
    
    time.sleep(1)
    
    # upload the sequence
    pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
        time.sleep(1)

    #print('Finished uploading', dictparams_oc["name"], '!')
         
    # make sure everything is finished (crucial)
    #time.sleep(1)#time.sleep(5)    
    
     # start the measurement
    #print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)
    
    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()
    
    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break

    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break
        
    #time.sleep(1)
            
    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    
    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)
    
    signal_data = pulsedmeasurementlogic.signal_data
    
    # calculate the fom
    up_norm = signal_data[1,0] / signal_data[1,0]
    down_norm = signal_data[1,1] / signal_data[1,0]
    
    signal_bench[kk]=np.real(1 - (up_norm - down_norm)) #pulsedmeasurementlogic.signal_data[1,0]
    time.sleep(20)
    

In [ ]:
plt.figure(14515151)
plt.plot(ampl_scale,signal_bench)
plt.show()

In [ ]:
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\signal_bench_oc_40ns_02042022_new.txt',signal_bench)
#np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\ampl_scale_oc_40ns_02042022_new.txt',ampl_scale)

## Benchmark Rectangular

In [ ]:
amp_scal_step=0.05
num_of_points=21

signal_bench=np.zeros(num_of_points)
ampl_scale=np.arange(1.0, -0.05, step=-amp_scal_step)

######################################################################################################
# Parameters and Settings
######################################################################################################

#FoM_path= r'C:\Users\Mesoscopic\Desktop\Redcrab_data\FoM'

# runtime of the experiment
runtime = 5

length_oc = 57.59e-9 / 2 #200e-9

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.28284e9
globaldict["rabi_period"] = 57.59e-9
globaldict["laser_length"] = 25e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 0)
pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

for kk in range (num_of_points):
    # predefined method
    dictparamsall = pulsedmasterlogic.generate_method_params
    ############################ Uploading all predefined methods needed ###################################################
    # predefined method
    dict_qst = dictparamsall["benchmark_rec_pulse"]
    dict_qst["name"] = 'recbenchmark'
    dict_qst["length"] = length_oc
    dict_qst["amp_scal_start"] = kk * 0.05
    dict_qst["amp_scal_step"] = amp_scal_step
    dict_qst["num_of_points"] = 1
    dict_qst["amp_rec"] = 0.5 #0.0331175

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('benchmark_rec_pulse',kwarg_dict = dict_qst)  
    
    time.sleep(1)
    
    # upload the sequence
    pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
        time.sleep(1)

    #print('Finished uploading', dictparams_oc["name"], '!')
         
    # make sure everything is finished (crucial)
    #time.sleep(1)#time.sleep(5)    
    
     # start the measurement
    #print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)
    
    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()
    
    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break

    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break
        
    #time.sleep(1)
            
    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    
    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)
    
    signal_data = pulsedmeasurementlogic.signal_data
    
    # calculate the fom
    up_norm = signal_data[1,0] / signal_data[1,0]
    down_norm = signal_data[1,1] / signal_data[1,0]
    
    signal_bench[kk]=np.real(1 - (up_norm - down_norm))
    time.sleep(20)
    

In [ ]:
pulsedmeasurementlogic.signal_data

In [ ]:
plt.figure(14515151)
plt.plot(ampl_scale,signal_bench)
plt.show()

In [ ]:
np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\signal_bench_rec_fast_01042022.txt',signal_bench)
np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\ampl_scale_rec_fast_01042022.txt',ampl_scale)

In [ ]:
ampl_scale=np.arange(1.0, 0, step=-amp_scal_step)

In [ ]:
test_oc = np.loadtxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\signal_bench_oc_210322_new.txt')
test_rec = np.loadtxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\signal_bench_rec_210322.txt')

plt.plot(test_oc)
plt.plot(test_rec)
plt.show()

In [ ]:
test_fom = np.loadtxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\results_state\fom_all_200ns_210322.txt')
plt.plot(test_fom)
plt.show()

In [ ]:
test_array = np.zeros(10)

In [ ]:
type(test_array)

In [ ]:
np.ndarray

In [ ]:
type(np.ndarray([0,1]))

In [ ]:
type(np.array([0,1]))

In [ ]:
test_array